In [1]:
#imports and set up

from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
from IPython.display import display
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import time
import sys
from process_pickle import process_pickle
%matplotlib inline
from ipywidgets import interact, interactive, fixed #Sliders for image selection
import ipywidgets as widgets
mpl.rc('figure',  figsize=(4.77, 2.95))
mpl.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
mpl.rcParams['lines.linewidth'] = 0.6
mpl.rcParams['lines.color'] = 'r'
plt.rc('grid', linestyle="--", color='gray')

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=FutureWarning)

import numpy as np
import pandas as pd
from pandas import DataFrame, Series  

import pims
import trackpy as tp
import os

In [2]:
# function that is called to configure how plots are to be drawn

def setup_plot():
    ax = plt.gca()
    SPINE_COLOR = 'gray'
    for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)

    for spine in ['left', 'bottom']:
        ax.spines[spine].set_color(SPINE_COLOR)
        ax.spines[spine].set_linewidth(0.5)

    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_tick_params(direction='out', color=SPINE_COLOR)
    plt.grid()
    fig = plt.gcf()
    fig.set_size_inches(4.77, 2.95, forward=True)

Use the file picker to locate the .pkl containing the particle trajectories

or select a previously output .xlsx to skip the calculations

**If it doesn't appear you probably need to minimise your browser or alt tab around for it**

In [3]:
set_custom_data_directory = True

In [4]:
# select the .pkl files you wish to analyse. When you're done selecting cancel should
# allow you to continue to the next cell.

import tkinter as tk
from tkinter import filedialog

pkl_to_process = []
file_path = "1"
while len(file_path) != 0:
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    if len(file_path) != 0:
        pkl_to_process.append(file_path)

In [5]:
# confirm you've selected the correct files

pkl_to_process

['C:/Users/lc16629/OneDrive - University of Bristol/Desktop/Python/DATA/22-1-18 0.8gL 25C (Cropped)/data output/trajectories.pkl']

In [6]:
calculate = True

In [7]:
# configure the calculations to be carried out on the data contained in the .pkl files
# if you don't enter "NEXT" at the end you can queue another set of calculations to be carried out on the 
# same .pkl, e.g with a frame lookback.
pkl_param = []

i = 0
mpp = 15 # microns per pixel
timestep = 30 # seconds between images

while i < len(pkl_to_process):
    print(pkl_to_process[i])
    print("Setting Parameters...")
    # uncomment the below lines to set mpp or timestep on a per .pkl basis
    #mpp = int(input("Microns per Pixel: "))
    #timestep = int(input("Time between frames in seconds: "))
    frame_lookback = int(input("How many frames to look back for velocity calculations? "))
    pkl_param.append([i,mpp,timestep,frame_lookback])
    move_next = input("Enter NEXT to move to next pkl, anything else to add another analysis run for the current particle")
    if move_next == "NEXT":
        i += 1

C:/Users/lc16629/OneDrive - University of Bristol/Desktop/Python/DATA/22-1-18 0.8gL 25C (Cropped)/data output/trajectories.pkl
Setting Parameters...
How many frames to look back for velocity calculations? 25
Enter NEXT to move to next pkl, anything else to add another analysis run for the current particleNEXT


In [8]:
pkl_param

[[0, 15, 30, 25]]

In [9]:
# for the sets of calculations you've requested, select folders to store the results for each set in.

data_dirs = []
data_dir = True
counter = 0
for i in pkl_param:
    root = tk.Tk()
    root.withdraw()
    data_dir = filedialog.askdirectory(title="Pick data dir for {} {}".format(pkl_to_process[i[0]], i[3]))
    data_dirs.append(data_dir)

In [10]:
# if the folders you've requested the results stored in don't exist, make them:

for directory in data_dirs:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
for c, param in enumerate(pkl_param):
    # the process_pickle function is contained within a separate file - process.py
    df = process_pickle(pkl_to_process[param[0]], data_dirs[c], param[1], param[2], param[3])

In [11]:
tstep = 30 # in Seconds - 60 for 1min, 30 for 30s

def vector_plot(general_directory):
    npstep = 10
    xdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 2])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x').values
    xdf = xdf/20
    xdf = np.rint(xdf)
    xdf = 20*xdf
    
    ydf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 1])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y').values
    ydf = ydf/20
    ydf = np.rint(ydf)
    ydf = 20*ydf

    xdf[np.isnan(xdf)] = 0
    ydf[np.isnan(ydf)] = 0
    
    vxdf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 16])
    vxdf = vxdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x_velocity').values

    vydf = pd.read_csv(general_directory + '/data.csv', engine='python', usecols=[9, 10, 15])
    vydf = vydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y_velocity').values
    nofrm = len(vxdf)
    noprt = len(vxdf[0])
    
    maxvalx = np.nanmax(vxdf)
    maxvaly = np.nanmax(vydf)
    
    if maxvalx >= maxvaly:
        maxval = maxvalx
    else:
        maxval = maxvaly
                
    xs = np.linspace(0, 800, 41)
    ys = np.linspace(0, 1600, 81)
    xi,yi = np.meshgrid(xs, ys)
    
    for f in range(0, nofrm-1, npstep):
        
        print('frame = %s' % f)
        
        vx_vals=np.full((ys.shape[0],xs.shape[0],noprt),np.nan)
        vy_vals=np.full((ys.shape[0],xs.shape[0],noprt),np.nan)

        for p in range(1, noprt-1):
                x = int((xdf[f, p])/20)
                y = int((ydf[f, p])/20)

                vx_vals[y,x,p] = (vxdf[f, p])/maxval
                vy_vals[y,x,p] = (vydf[f, p])/maxval

        vx_vals=np.nanmean(vx_vals, axis=2)
        vy_vals=np.nanmean(vy_vals, axis=2)

        if not os.path.exists(general_directory + "/vector_frames/"):
            os.makedirs(general_directory + "/vector_frames/")  

        fig = plt.figure(figsize=(5,13))
        ax = fig.add_subplot(1, 1, 1)
        ax.set_xlim(0, 500)
        ax.set_ylim(1300, 0)
        
        vx = vx_vals[:,:]
        vy = vy_vals[:,:]
        
        ax.quiver(xi, yi, vx, vy, alpha=0.5)
        ax.quiver(xi, yi, vx, vy, edgecolor='k', facecolor='none', linewidth=.5)
        plt.title("t = " + (str(round(((f*tstep)/3600), 1))) + " hrs")
        
        plt.savefig(general_directory + "/vector_frames/vector_" + str(f) + ".png")
        plt.close()
        
    vecim = []
    for i in range(0, nofrm-1, npstep):
        vecim.append(imageio.imread(general_directory + "/vector_frames/vector_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/vector_frames/vector.gif"), vecim)
                          
for c, param in enumerate(pkl_param):
    import imageio
    from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
    print('\n')
    print('Processing data: ')
    print('\n')
    vector_plot(data_dirs[c])
    print('\n')
    print('Done!')



Processing data: 


frame = 0


C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: Mean of empty slice
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: Mean of empty slice
C:\Program Files\Anaconda3\lib\site-packages\matplotlib\font_manager.py:1241: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


frame = 10
frame = 20
frame = 30
frame = 40
frame = 50
frame = 60
frame = 70
frame = 80
frame = 90
frame = 100
frame = 110
frame = 120
frame = 130
frame = 140
frame = 150
frame = 160
frame = 170
frame = 180
frame = 190
frame = 200
frame = 210
frame = 220
frame = 230
frame = 240
frame = 250
frame = 260
frame = 270
frame = 280
frame = 290
frame = 300
frame = 310
frame = 320
frame = 330
frame = 340
frame = 350
frame = 360
frame = 370
frame = 380
frame = 390
frame = 400
frame = 410
frame = 420
frame = 430
frame = 440
frame = 450
frame = 460
frame = 470
frame = 480
frame = 490
frame = 500
frame = 510
frame = 520
frame = 530
frame = 540
frame = 550
frame = 560
frame = 570
frame = 580
frame = 590
frame = 600
frame = 610
frame = 620
frame = 630
frame = 640
frame = 650
frame = 660
frame = 670
frame = 680
frame = 690
frame = 700
frame = 710
frame = 720
frame = 730
frame = 740
frame = 750
frame = 760
frame = 770
frame = 780
frame = 790
frame = 800
frame = 810
frame = 820
frame = 830
frame = 840
f

frame = 6400
frame = 6410
frame = 6420
frame = 6430
frame = 6440
frame = 6450
frame = 6460
frame = 6470
frame = 6480
frame = 6490
frame = 6500
frame = 6510
frame = 6520
frame = 6530
frame = 6540
frame = 6550
frame = 6560
frame = 6570
frame = 6580
frame = 6590
frame = 6600
frame = 6610
frame = 6620
frame = 6630
frame = 6640
frame = 6650
frame = 6660
frame = 6670
frame = 6680
frame = 6690
frame = 6700
frame = 6710
frame = 6720
frame = 6730
frame = 6740
frame = 6750
frame = 6760
frame = 6770
frame = 6780
frame = 6790
frame = 6800
frame = 6810
frame = 6820
frame = 6830
frame = 6840
frame = 6850
frame = 6860
frame = 6870
frame = 6880
frame = 6890
frame = 6900
frame = 6910
frame = 6920
frame = 6930
frame = 6940
frame = 6950
frame = 6960
frame = 6970
frame = 6980
frame = 6990
frame = 7000
frame = 7010
frame = 7020
frame = 7030
frame = 7040
frame = 7050
frame = 7060
frame = 7070
frame = 7080
frame = 7090
frame = 7100
frame = 7110
frame = 7120
frame = 7130
frame = 7140
frame = 7150
frame = 7160

In [ ]:
def hetero_scaled_map(general_directory):
    npstep = 50
    xdf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 2])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x').values
    xdf = xdf/50
    xdf = np.rint(xdf)
    xdf = 50*xdf
    
    ydf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 1])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y').values
    ydf = ydf/50
    ydf = np.rint(ydf)
    ydf = 50*ydf
    
    xdf[np.isnan(xdf)] = 0
    ydf[np.isnan(ydf)] = 0
   
    
    
    vxdf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 16])

    vxdf = vxdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='x_velocity').values

    vydf = pd.read_csv(general_directory + '/data.csv', usecols=[9, 10, 15])
    vydf = vydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       values='y_velocity').values
    
    nofrm = len(vxdf)
    noprt = len(vxdf[0])
    hetdf = np.empty((nofrm,noprt,))
    hetdf[:] = np.nan
    
    for p in range(1, noprt-1): # Chi-squared and heterogeneity type test for each particle in each frame
        for f in range(1, nofrm-1):
            Ex = vxdf[(f-1),p]
            Ox = vxdf[f,p]
            Ey = vydf[(f-1),p]
            Oy = vydf[f,p]
            
            chix = abs(((Ox-Ex)**2)/(Ex)) # Chi squared test for Vx
            chiy = abs(((Oy-Ey)**2)/(Ey)) # Chi squared test for Vy
            chi = chix + chiy
            hetdf[f,p] = chi # I2 heterogeneity value, for 2 degrees of freedom
            
    #hetdf = np.log10(hetdf)
    #hetdf = (hetdf + abs(np.nanmin(hetdf)))
    vmx = np.nanmax(hetdf)
    vmn = np.nanmin(hetdf)
    
    xs = np.linspace(0, 800, 17)
    ys = np.linspace(0, 1600, 33)
    xi,yi = np.meshgrid(xs, ys)    
        
    I_vals=np.full((ys.shape[0],xs.shape[0],nofrm,noprt),np.nan)
    
    for p in range(1, noprt-1):
        for f in range(0, nofrm-1):
            x = int((xdf[f, p])/50)
            y = int((ydf[f, p])/50)
                            
            I_vals[y,x,f,p] = hetdf[f, p]
    
    I_vals=np.nanmean(I_vals, axis=3)
    
    I_vals[np.isnan(I_vals)] = 0
    
    print(I_vals)
    
    if not os.path.exists(general_directory + "/hetero_map/"):
        os.makedirs(general_directory + "/hetero_map/")

    for i in range(0, nofrm-1, npstep): 
    
        heatfig = plt.figure(figsize=(5,13))
        axh = heatfig.add_subplot(1, 1, 1)
        axh.set_xlim(0, 500)
        axh.set_ylim(0, 1300)       
        
        im = plt.contourf(xi, yi, I_vals[:,:,i], cmap='RdBu_r')
        plt.title("t = " + (str(i*30)) + "s") 
        im.set_clim(vmax=vmx, vmin=vmn)
        
        divider = make_axes_locatable(axh)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax, orientation='vertical')
        
        plt.savefig(general_directory + "/hetero_map/img_" + str(i) + ".png")
        plt.close()
        
    heatim = []
    for i in range(0, nofrm-1, npstep):
        heatim.append(imageio.imread(general_directory + "/hetero_map/img_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/hetero_map/anim.gif"), heatim)
    
for c, param in enumerate(pkl_param):
    import imageio
    from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
    print('\n')
    print('Processing data: ')
    print('\n')
    hetero_scaled_map(data_dirs[c])
    print('\n')
    print('Done!')